In [1]:
from PIL import ImageGrab, ImageOps, Image
import pyautogui
import time
import numpy as np

In [2]:
def pics():
    try:
        dino = pyautogui.locateOnScreen('./images/dino_day.png')
        hi_score = pyautogui.locateOnScreen('./images/HI_day.png')
        state_coord = (dino[0], hi_score[1] + hi_score[3], hi_score[0] + hi_score[2], dino[1] + dino[3])
    except:
        dino = pyautogui.locateOnScreen('./images/dino_day_1.png')
        hi_score = pyautogui.locateOnScreen('./images/HI_day.png')
        state_coord = (dino[0], hi_score[1] + hi_score[3], hi_score[0] + hi_score[2], dino[1] + dino[3])
    return state_coord

class Coordinates():
    imediate_box = (214,552,500,689) #obstacles right ahead
    long_box = (214,552,1919,689) #everything ahead
    sky_box = (2,68,7,71) # check if it's day or night
    state_area = pics()

def restartGame():
    try: #Try to find the replay button
        replay_x, replay_y = pyautogui.locateCenterOnScreen('./images/replay_day.png')
    except:
        replayt_x, replay_y = pyautogui.locateCenterOnScreen('./images/replay_night.png')
    pyautogui.click(replay_x, replay_y)
    
def is_alive():
    try:
        replay_x, replay_y = pyautogui.locateCenterOnScreen('./images/replay_day.png')
        #print('dead day')
        alive = 0
    except:
        try:
            replay_x, replay_y = pyautogui.locateCenterOnScreen('./images/replay_night.png')
            #print('dead night')
            alive = 0
        except:
            alive = 1
            #print('alive')
    return alive

def pressSpace():
    pyautogui.keyDown('space')
    time.sleep(.05)
    pyautogui.keyUp('space')

def state_data():
    # First check eveything ahead:
    image_1 = ImageGrab.grab(Coordinates.state_area)
    grayImage_1 = ImageOps.grayscale(image_1)
    num_1 = np.sum(np.array(grayImage_1.getcolors())) #reduce all the information to one number (major simplification)
    # Check eveything ahead twice to account for speed
    image_2 = ImageGrab.grab(Coordinates.state_area)
    grayImage_2 = ImageOps.grayscale(image_2)
    num_2 = np.array(grayImage_2.getcolors())
    time.sleep(.05)
    image_3 = ImageGrab.grab(Coordinates.state_area)
    grayImage_3 = ImageOps.grayscale(image_3)
    num_3 = np.array(grayImage_3.getcolors())
    dif = np.sum(num_2)-np.sum(num_3)
    # Check the sky to see if it's day or night
    image_4 = ImageGrab.grab(Coordinates.state_area)
    grayImage_4 = ImageOps.grayscale(image_4)
    num_4 = np.sum(np.array(grayImage_4.getcolors()))
    return num_1, dif, num_4
    
def find_survivors(gens, names):
    payoffs = {} #species' results
    survivors = {} #survivor data
    for i in range(4):
        payoffs[names[i]] = len(gens[names[i]]) # the payoff is going to be the number of states discovered since the ones that know more states have lived the longest
    listed = [payoffs[names[i]] for i in range(4)]
    payoff_vals = sorted(listed)
    sur_pay = payoff_vals[-2:] #we'll have two survivors and so we need the two highest payoffs
    sur_spec = [i for i, x in enumerate(listed) if x in sur_pay] #identifiying the survivors
    return sur_spec

In [3]:
def play(gens,state_space,names,start_time):
    payoffs = {}
    for i in range(4):
        #print('spec')
        #print(i)
        restartGame()
        while True:
            # We start by cheching the current state
            current_state = (state_data()[0], state_data()[1], state_data()[2])
            #print('cur st')
            #print(current_state)
            # If it's an unkown state to the current species add it to it's state space
            if current_state not in state_space[names[i]]:
                state_space[names[i]] += [current_state]
                #print('st sp')
                #print(state_space)
                # Let's pick a policy for each species when it encounters a new state
                if i == 0:
                    gens[names[i]] += [0] #first species will always pick 0
                if i == 1:
                    gens[names[i]] += [1] #second species will always pick 1
                if i == 2:
                    gens[names[i]] += list(np.random.choice([0, 1], size=(1), p=[1/3, 2/3])) #third species will pick 1 with probability 2/3
                if i == 3:
                    gens[names[i]] += list(np.random.choice([0, 1], size=(1), p=[2/3, 1/3]))
                #print('n st sp')
                #print(state_space)
                #print('gens')
                #print(gens)
            state_pos = state_space[names[i]].index(current_state) #find the position of the current state
            if gens[names[i]][state_pos] == 1 and is_alive() == 1: #if the policy for the current state is 1, jump!
                pressSpace()
            elif is_alive() == 0:
                #print('dead')
                break
    return gens, state_space

In [4]:
def new_generation(gens, survivors, names, state_space):
    new_gens = {}
    new_state_space = {}
    survivors = survivors[:2]
    size_1 = len(gens[names[survivors[0]]])
    size_2 = len(gens[names[survivors[1]]])
    cross_num = int(np.ceil(size_2/5)) # at lead 20% crossover
    pos_1 = np.random.choice(size_1, cross_num, replace=False) #gene positions to be swapped of the first survivor's first child
    pos_2 = np.random.choice(size_1, cross_num, replace=False) #gene positions to be swapped of the first survivor's second child
    pos_3 = np.random.choice(size_2, cross_num, replace=False) #same for the second survivor...
    pos_4 = np.random.choice(size_2, cross_num, replace=False)
    for i in range(2):
        new_gens[names[2*i]] = gens[names[survivors[i]]].copy() #we start by making the children the copy of their parents
        new_state_space[names[2*i]] = state_space[names[survivors[i]]].copy() #need to copy the state space as well as we're insterested in the relationship between state and action
        new_gens[names[2*i+1]] = gens[names[survivors[i]]].copy()
        new_state_space[names[2*i+1]] = state_space[names[survivors[i]]].copy()
    #crossover:
    for k in range(cross_num):
        new_gens[names[0]][pos_1[k]] = gens[names[survivors[1]]].copy()[pos_3[k]] #the new genes for specie 1, pos_1 is going to be swapped with the genes from survivor 2 pos_3
        new_state_space[names[0]][pos_1[k]] = state_space[names[survivors[1]]].copy()[pos_3[k]] #states must also be swapped since the policy is a relationship between state and action
        
        new_gens[names[1]][pos_2[k]] = gens[names[survivors[1]]].copy()[pos_4[k]]
        new_state_space[names[1]][pos_2[k]] = state_space[names[survivors[1]]].copy()[pos_4[k]]
        
        new_gens[names[2]][pos_3[k]] = gens[names[survivors[0]]].copy()[pos_1[k]]
        new_state_space[names[2]][pos_3[k]] = state_space[names[survivors[0]]].copy()[pos_1[k]]
        
        new_gens[names[3]][pos_4[k]] = gens[names[survivors[0]]].copy()[pos_2[k]]
        new_state_space[names[3]][pos_4[k]] = state_space[names[survivors[0]]].copy()[pos_2[k]]
    #mutations:
    mut_num = int(np.ceil(size_2/20)) #at least 5% mutation
    mut_pos_1 = np.random.choice(int(np.minimum(size_1,size_2)), mut_num, replace=False) #randomly choose positions to chance policy
    for l in mut_pos_1:
        if new_gens[names[0]][l] == 0: #If the value was 0, turn it into 1
            new_gens[names[0]][l] = 1
        else:
            new_gens[names[0]][l] = 0 #Otherwise turn it into 0
    mut_pos_2 = np.random.choice(int(np.minimum(size_1,size_2)), mut_num, replace=False)
    for l in mut_pos_2:
        if new_gens[names[1]][l] == 0: # Se era 0 vira 1
            new_gens[names[1]][l] = 1
        else:
            new_gens[names[1]][l] = 0
    mut_pos_3 = np.random.choice(int(np.minimum(size_1,size_2)), mut_num, replace=False)
    for l in mut_pos_3:
        if new_gens[names[2]][l] == 0: # Se era 0 vira 1
            new_gens[names[2]][l] = 1
        else:
            new_gens[names[2]][l] = 0
    mut_pos_4 = np.random.choice(int(np.minimum(size_1,size_2)), mut_num, replace=False)
    for l in mut_pos_4:
        if new_gens[names[3]][l] == 0: # Se era 0 vira 1
            new_gens[names[3]][l] = 1
        else:
            new_gens[names[3]][l] = 0
    return new_gens, new_state_space

In [5]:
start_time = time.time()
state_space = {} #espaço de estados conhecidos da espécie i
gens = {} #espaço de ações escolhidas pela espécie i
names = ['spec_%s' %(str(i)) for i in range (1,5)] #nomes das espécies
#criando as políticas das espécies originais
for i in range(4):
    gens[names[i]] = []
    state_space[names[i]] = []
print('gens before game')
print(gens)
#rodando as espécies:
while True:
    gens, state_space = play(gens,state_space,names,start_time) #achar as relações entre estado e ações de cada espécie
    print('gens after game')
    print(gens)
    survivors = find_survivors(gens, names) #achar os sobreviventes
    print('survivors') 
    print(survivors)
    gens, state_space = new_generation(gens, survivors, names, state_space) #nova geração
    print('new generation gens')
    print(gens)
    if time.time() - start_time > 60: #tempo máximo do loop
        break

gens before game
{'spec_1': [], 'spec_2': [], 'spec_3': [], 'spec_4': []}
gens after game
{'spec_1': [0], 'spec_2': [1, 1], 'spec_3': [0], 'spec_4': [0]}
survivors
[0, 1, 2, 3]
new generation gens
{'spec_1': [0], 'spec_2': [0], 'spec_3': [0, 0], 'spec_4': [0, 0]}
gens after game
{'spec_1': [0, 0], 'spec_2': [0, 1], 'spec_3': [0, 0], 'spec_4': [0, 0]}
survivors
[0, 1, 2, 3]
new generation gens
{'spec_1': [1, 0], 'spec_2': [0, 1], 'spec_3': [1, 1], 'spec_4': [0, 0]}
gens after game
{'spec_1': [1, 0, 0], 'spec_2': [0, 1, 1], 'spec_3': [1, 1, 1], 'spec_4': [0, 0]}
survivors
[0, 1, 2]
new generation gens
{'spec_1': [0, 1, 0], 'spec_2': [1, 1, 1], 'spec_3': [0, 1, 1], 'spec_4': [0, 0, 0]}
